In [ ]:
# install pytorch 
# !pip install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U

In [4]:
# test torch version and CUDA device
import torch
print(torch.__version__)
a = torch.Tensor([1]).cuda()
print(a)

1.9.0
tensor([1.], device='cuda:0')


In [5]:
# mount google drive
# from google.colab import drive
# drive.mount("/content/gdrive")

In [113]:
# # change to project home directory
# import os
# os.chdir("/content/gdrive/My Drive/object_detection_frcnn_mscoco_boilerplate")

In [6]:
pwd

'/home/hasan/Downloads/dark_pattern_project/DLDarkPatterns/object_detection/object_detection_frcnn_mscoco_boilerplate'

In [7]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [8]:
import utils


In [14]:
import pprint
pp = pprint.PrettyPrinter()
import json

In [15]:
# path to data and annotation
data_dir = 'inference_data'

In [17]:
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      #transforms.Normalize([0.485, 0.456, 0.406],
                                      #                     [0.229, 0.224, 0.225])
                                     ])
dataset_test = datasets.ImageFolder(data_dir, transform=test_transforms)

In [18]:
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)



In [19]:
# select device (whether GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
num_classes = 6

In [21]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load('trained-visual-cue-detection-model.pth'))

<All keys matched successfully>

In [22]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [23]:
print("================= INFERENCE ===========================")
with torch.no_grad():
    correct = 0
    total = 0
    inference_result = {}
    # for images, labels in data_loader_test:
    for i, (images, labels) in enumerate(data_loader_test, 0):
        print("================== batch ==================")
        sample_fname, _ = data_loader_test.dataset.samples[i]
        filename = sample_fname.split("/")[2]
        print("filename: ", filename)
        outputs = model(images)
        print("outputs")
        pp.pprint(outputs)

        # init boxes, labels, scores
        boxes = None
        labels = None
        scores = None
        
        # get boxes, labels, scores
        if len(outputs[0]["boxes"]) != 0:
          boxes = outputs[0]["boxes"][0].numpy().tolist()
        if len(outputs[0]["labels"]) != 0:
          labels = outputs[0]["labels"][0].numpy().tolist()
        if len(outputs[0]["scores"]) != 0:
          scores = outputs[0]["scores"][0].numpy().tolist()
        
        # populate JSON
        inference_result["boxes"] = boxes
        inference_result["labels"] = labels
        inference_result["scores"] = scores

        print("inference result")
        pp.pprint(inference_result)

        # write inference result
        inference_output_dir = "./inference_output/"
        inference_output_filename = inference_output_dir + filename.split(".")[0] + ".json"
        with open(inference_output_filename, "w") as fp:
          json.dump(inference_result, fp)
        fp.close()
        # break

================= INFERENCE ===========================
================== batch ==================
filename:  communication_1--messenger-text-and-video-1-25_9979.jpg


/home/hasan/anaconda3/envs/dl_dp_obj_det_env/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


outputs
[{'boxes': tensor([[138.6384,  66.6878, 182.4153, 138.3146]]),
  'labels': tensor([3]),
  'scores': tensor([0.2108])}]
inference result
{'boxes': [138.6383819580078,
           66.68780517578125,
           182.41531372070312,
           138.31463623046875],
 'labels': 3,
 'scores': 0.21083638072013855}


In [ ]:
# # write inference result
# with open('./inference_result.json', 'w') as fp:
#   json.dump(inference_result, fp)